In [15]:
import h2o
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator

In [16]:
h2o.init()

H2O cluster uptime:,5 minutes 50 seconds 603 milliseconds
H2O cluster version:,3.5.0.99999
H2O cluster name:,ludirehak
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [17]:
from h2o.h2o import _locate # private function. used to find files within h2o git project directory.

# Import walking gait data
gait = h2o.upload_file(path=_locate("smalldata/glrm_test/subject01_walk1.csv"))
gait.describe()


Parse Progress: [##################################################] 100%
Uploaded pyf9354afc-0978-42fa-8c42-4774d23d80d3 into cluster with 151 rows and 124 cols
Rows: 151 Cols: 124

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C2S,2-Byte Fractions,1,0.8,386 B,0.5
C4S,4-Byte Fractions,123,99.19355,82.6 KB,99.54594



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.38:54321,83.0 KB,151.0,1.0,124.0
mean,83.0 KB,151.0,1.0,124.0
min,83.0 KB,151.0,1.0,124.0
max,83.0 KB,151.0,1.0,124.0
stddev,0 B,0.0,0.0,0.0
total,83.0 KB,151.0,1.0,124.0



Column-by-Column Summary:



,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.6,107.79163,600.01556,1028.7,-152.5,399.40009,1034.9,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.3,550.69415,684.75494,-133.588,435.01773,627.49988,-170.9,319.34705,444.43289,157.69023,376.44724,403.25574,102.54192,293.89526,368.28577,165.6133,341.2023,438.80435,-173.0,365.72482,375.95242,-121.0,304.46347,393.67728,-173.4,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,58.7473,-74.0361,211.85968,69.02702,-69.30095,185.39088,48.22646,-140.1,254.42706,36.65646,-88.18266,627.38177,1342.9,-35.20795,511.80032,1427.4,164.48206,551.72424,1453.6,-249.9,485.94559,1227.9,216.05853,531.12134,1217.6,-266.6,424.32767,1129.1,242.29697,460.03223,1123.8,-304.8,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.6,502.82971,886.58942,-300.9,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,199.80693,36.02512,-153.1,228.04898,42.9706,-54.88903,675.97009,1562.3,44.15432,683.50897,1567.3,-94.07465,614.13971,1740.4,-19.48838
maxs,2.5,641.12048,1091.4,180.07411,661.46918,1078.0,-80.37424,451.5914,1077.8,55.36993,692.89514,788.11365,217.07512,766.44531,750.72375,136.25021,728.67242,672.42242,209.89946,674.53168,783.71576,-138.1,773.97705,780.81445,-66.30186,713.16107,681.38623,-123.1844,719.69702,531.05676,200.56787,799.47015,457.5513,154.56749,751.88885,458.64368,202.59061,764.30676,494.35605,-116.5,821.66901,425.57733,-63.51654,750.77588,473.02411,-124.5,812.50183,320.70309,108.161,935.61414,198.72064,86.30818,934.44775,211.37376,188.70589,1073.9,179.28133,153.59363,821.7641,321.80249,-34.92287,971.31763,178.00603,-6.83138,955.73096,184.36398,-10

In [18]:
# Plot first row of data on x- vs. y-coordinate features
gait_row = gait[1,:].drop("Time")
gait_row_np = np.array(h2o.as_list(gait_row))
x_coords = range(0, gait_row_np.shape[1], 3)
y_coords = range(1, gait_row_np.shape[1], 3)

x_pts = gait_row_np[0,x_coords]
y_pts = gait_row_np[0,y_coords]
plt.plot(x_pts, y_pts, 'bo')

# Add feature labels to each point
feat_names = [nam[:-2] for nam in gait_row.col_names[1::3]]
for i in xrange(len(feat_names)):
    plt.annotate(feat_names[i], xy = [x_pts[i], y_pts[i]])
plt.title("Location of Body Parts at Time 0")
plt.xlabel("X-Coordinate Weight")
plt.ylabel("Y-Coordinate Weight")
plt.show()

In [19]:
# Basic GLRM using quadratic loss and no regularization (PCA)
model = H2OGeneralizedLowRankEstimator(k=10, loss="Quadratic", regularization_x="None", regularization_y="None", max_iterations=1000)
model.train(x=range(1,gait.ncol), training_frame=gait)
model.show()


glrm Model Build Progress: [##################################################] 100%
Model Details
H2OGeneralizedLowRankEstimator :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1445562467796_5

Model Summary:


,number_of_iterations,final_step_size,final_objective_value
,15.0,0.0,338916.5




ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 338916.511217
Misclassification Error (Categorical): 0.0

Scoring History:


,timestamp,duration,iteration,step_size,objective
,2015-10-22 18:13:41,0.064 sec,0.0,1.05,345883.2
,2015-10-22 18:13:41,0.073 sec,1.0,0.7,345883.2
,2015-10-22 18:13:41,0.082 sec,2.0,0.5,345883.2
,2015-10-22 18:13:41,0.097 sec,3.0,0.3,345883.2
,2015-10-22 18:13:41,0.106 sec,4.0,0.2,345883.2
,2015-10-22 18:13:41,0.118 sec,5.0,0.1,345883.2
,2015-10-22 18:13:41,0.129 sec,6.0,0.0,345883.2
,2015-10-22 18:13:41,0.141 sec,7.0,0.0,345883.2
,2015-10-22 18:13:41,0.150 sec,8.0,0.0,338916.5
,2015-10-22 18:13:41,0.159 sec,9.0,0.0,338916.5


In [20]:
# Plot objective function value each iteration
model_score = model.score_history()
plt.xlabel("Iteration")
plt.ylabel("Objective")
plt.title("Objective Function Value per Iteration")
print model_score
plt.plot(model_score["iteration"], model_score["objective"])
plt.show()

                timestamp    duration  iteration  step_size      objective
0     2015-10-22 18:13:41   0.064 sec          0   1.050000  345883.210313
1     2015-10-22 18:13:41   0.073 sec          1   0.700000  345883.210313
2     2015-10-22 18:13:41   0.082 sec          2   0.466667  345883.210313
3     2015-10-22 18:13:41   0.097 sec          3   0.311111  345883.210313
4     2015-10-22 18:13:41   0.106 sec          4   0.155556  345883.210313
5     2015-10-22 18:13:41   0.118 sec          5   0.051852  345883.210313
6     2015-10-22 18:13:41   0.129 sec          6   0.012963  345883.210313
7     2015-10-22 18:13:41   0.141 sec          7   0.002593  345883.210313
8     2015-10-22 18:13:41   0.150 sec          8   0.002722  338916.504756
9     2015-10-22 18:13:41   0.159 sec          9   0.001815  338916.504756
10    2015-10-22 18:13:41   0.169 sec         10   0.001210  338916.504756
11    2015-10-22 18:13:41   0.178 sec         11   0.000807  338916.504756
12    2015-10-22 18:13:41

In [21]:
# Archetype to feature mapping (Y)
gait_y = model._model_json["output"]["archetypes"]
print gait_y

gait_y_np = np.array(model.archetypes())
x_coords = range(0, gait_y_np.shape[1], 3)
y_coords = range(1, gait_y_np.shape[1], 3)

# Plot archetypes on x- vs. y-coordinate features
for k in xrange(gait_y_np.shape[0]):
    x_pts = gait_y_np[k, x_coords]
    y_pts = gait_y_np[k, y_coords]
    plt.plot(x_pts, y_pts, 'bo')

    # Add feature labels to each point
    feat_names = [nam[:-1] for nam in gait_y.col_header[1::3]]
    for i in xrange(len(feat_names)):
        plt.annotate(feat_names[i], xy = [x_pts[i], y_pts[i]])
    plt.title("Feature Weights of Archetype " + str(k+1))
    plt.xlabel("X-Coordinate Weight")
    plt.ylabel("Y-Coordinate Weight")
    plt.show()


Archetypes:


,rasisx,rasisy,rasisz,lasisx,lasisy,lasisz,vsacralx,vsacraly,vsacralz,rthighupperx,rthighuppery,rthighupperz,rthighfrontx,rthighfronty,rthighfrontz,rthighrearx,rthighreary,rthighrearz,lthighupperx,lthighuppery,lthighupperz,lthighfrontx,lthighfronty,lthighfrontz,lthighrearx,lthighreary,lthighrearz,rshankupperx,rshankuppery,rshankupperz,rshankfrontx,rshankfronty,rshankfrontz,rshankrearx,rshankreary,rshankrearz,lshankupperx,lshankuppery,lshankupperz,lshankfrontx,lshankfronty,lshankfrontz,lshankrearx,lshankreary,lshankrearz,rheelx,rheely,rheelz,rmidfootsupx,rmidfootsupy,rmidfootsupz,rmidfootlatx,rmidfootlaty,rmidfootlatz,rtoetipx,rtoetipy,rtoetipz,lheelx,lheely,lheelz,lmidfootsupx,lmidfootsupy,lmidfootsupz,lmidfootlatx,lmidfootlaty,lmidfootlatz,ltoetipx,ltoetipy,ltoetipz,sternumx,sternumy,sternumz,racromiumx,racromiumy,racromiumz,lacromiumx,lacromiumy,lacromiumz,rbicepx,rbicepy,rbicepz,lbicepx,lbicepy,lbicepz,relbowx,relbowy,relbowz,lelbowx,lelbowy,lelbowz,rwristmedx,rwristmedy,rwristmedz,rwristlatx,rwristlaty,rwristlatz,lwristmedx,lwristmedy,lwristmedz,lwristlatx,lwristlaty,lwristlatz,rtoelatx,rtoelaty,rtoelatz,rtoemedx,rtoemedy,rtoemedz,ltoelatx,ltoelaty,ltoelatz,ltoemedx,ltoemedy,ltoemedz,rtemplex,rtempley,rtemplez,ltemplex,ltempley,ltemplez,topheadx,topheady,topheadz
Arch1,597.8,1064.7,179.0,610.0,1058.2,-81.9,405.6,1065.3,48.3,519.4,746.4,214.5,573.3,678.5,121.2,493.2,627.9,199.9,636.4,761.5,-140.8,731.3,754.9,-74.7,676.1,659.5,-133.7,391.9,464.8,182.4,461.2,415.4,129.8,387.5,377.4,185.0,717.5,462.5,-128.5,779.4,423.0,-83.8,704.2,416.4,-134.9,250.0,80.1,89.7,395.1,78.8,62.1,383.2,65.6,169.0,536.2,44.9,110.5,723.0,78.6,-43.0,886.3,117.1,-36.3,860.7,106.1,-130.8,994.9,117.4,-78.9,636.0,1372.7,36.8,544.4,1460.4,251.1,556.5,1486.1,-166.5,578.4,1259.6,297.4,538.1,1251.5,-203.8,556.0,1152.5,331.5,470.2,1165.8,-238.3,765.1,992.8,282.5,742.5,961.6,335.2,571.3,933.6,-203.3,520.5,924.5,-258.5,468.3,38.4,187.9,478.2,43.1,79.0,925.6,95.7,-147.1,951.3,105.6,-48.0,694.0,1610.2,124.1,701.4,1615.9,-14.8,667.0,1780.9,63.6
Arch2,608.6,1054.0,117.4,609.8,1040.1,-143.6,411.9,1050.8,-1.8,652.7,747.2,172.4,726.5,705.8,76.9,687.9,631.0,168.8,479.1,758.7,-189.4,561.7,698.4,-131.6,448.1,653.4,-168.8,687.0,453.2,159.1,775.9,445.3,113.2,728.3,377.4,170.8,363.2,449.9,-160.6,399.1,387.9,-111.8,330.4,415.3,-165.2,772.0,60.0,92.9,904.1,119.7,76.3,894.7,97.6,180.5,1043.3,143.5,131.9,169.9,104.4,-62.5,338.7,77.3,-45.3,319.9,68.2,-132.7,440.8,45.8,-62.7,642.7,1351.5,-15.9,537.4,1445.1,188.7,591.0,1468.2,-224.2,513.5,1248.9,234.8,629.3,1235.4,-251.9,453.1,1153.7,259.6,597.9,1131.9,-293.3,579.6,916.0,269.8,532.5,903.6,313.7,778.2,972.0,-214.2,747.9,937.4,-273.9,984.2,105.1,203.8,997.0,123.0,96.0,373.2,38.0,-139.6,395.1,47.3,-38.6,718.5,1569.1,71.7,735.2,1573.5,-66.1,736.6,1746.7,13.0
Arch3,613.9,1079.7,176.1,614.4,1074.4,-85.3,414.9,1075.3,53.0,578.3,755.0,216.1,639.3,695.1,121.4,570.9,632.5,203.5,641.8,777.2,-151.2,745.9,770.5,-98.8,681.4,674.2,-144.6,498.8,459.8,185.9,572.8,419.0,132.7,504.4,373.0,189.4,668.0,488.8,-144.2,690.5,422.0,-95.5,628.8,461.4,-147.7,399.8,72.5,94.8,544.2,78.7,67.0,531.9,62.9,173.5,685.9,48.8,113.9,431.9,209.3,-41.7,575.1,128.8,-34.9,547.7,131.9,-128.5,653.2,57.7,-72.6,640.1,1389.0,40.2,548.6,1472.1,257.5,558.9,1506.1,-161.7,574.8,1269.1,301.0,553.2,1269.1,-202.0,544.9,1162.3,332.3,491.2,1180.7,-239.2,731.7,969.9,302.8,699.2,944.7,354.0,587.5,945.6,-207.5,539.6,939.6,-265.4,619.0,38.1,191.2,630.2,46.3,81.2,586.3,80.5,-142.3,615.6,79.8,-42.9,698.4,1621.1,134.2,707.4,1629.3,-4.9,679.9,1795.9,77.1
Arch4,625.9,1079.6,128.6,643.6,1068.2,-131.5,436.1,1069.6,-4.6,629.1,757.5,194.3,712.6,702.7,118.4,643.0,636.5,196.6,617.4,761.9,-178.2,708.1,735.1,-111.3,626.6,653.7,-157.9,552.4,516.2,183.9,585.1,431.2,142.1,498.2,448.3,187.8,628.2,440.4,-148.7,673.4,391.7,-91.7,607.6,397.2,-154.9,239.7,306.2,76.9,330.5,189.7,72.2,290.7,195.4,170.6,376.3,61.9,132.2,532.0,75.8,-52.6,701.6,74.9,-23.5,686.0,68.8,-120.8,813.3,50.4,-52.7,656.2,1381.8,-25.3,548.4,1

In [22]:
# Projection into archetype space (X)
x_key = model._model_json["output"]["representation_name"]
gait_x = h2o.get_frame(x_key)
gait_x.show()

time_np = np.array(h2o.as_list(gait["Time"]))
gait_x_np = np.array(h2o.as_list(gait_x))

# Plot archetypes over time
lines = []
for i in xrange(gait_x_np.shape[1]):
    lines += plt.plot(time_np, gait_x_np[:,i], '-')
plt.title("Archetypes over Time")
plt.xlabel("Time")
plt.ylabel("Archetypal Projection")
plt.legend(lines, gait_x.col_names)
plt.show()

H2OFrame with 151 rows and 10 columns: 


,Arch1,Arch2,Arch3,Arch4,Arch5,Arch6,Arch7,Arch8,Arch9,Arch10
0,1.132704,0.387077,-1.041831,-0.485789,0.752806,-0.234696,0.080678,1.026057,-0.461999,-0.142426
1,0.913236,0.356854,-0.852937,-0.514801,0.622235,-0.212839,0.088337,1.144347,-0.406937,-0.126244
2,0.708520,0.318724,-0.676550,-0.544785,0.484920,-0.208563,0.096793,1.273791,-0.331545,-0.111166
3,0.539707,0.281940,-0.523285,-0.551700,0.367090,-0.221301,0.100310,1.380833,-0.263246,-0.101216
4,0.409526,0.243284,-0.401218,-0.545490,0.285280,-0.262595,0.109814,1.478938,-0.220086,-0.089475
5,0.308033,0.216094,-0.306869,-0.493988,0.211845,-0.293279,0.101047,1.523479,-0.164632,-0.094548
6,0.255902,0.202039,-0.235221,-0.409895,0.167245,-0.354948,0.086692,1.537114,-0.134313,-0.108548
7,0.232645,0.187327,-0.205273,-0.288777,0.150907,-0.406792,0.064703,1.503912,-0.113875,-0.119711
8,0.248342,0.167704,-0.219963,-0.139003,0.161010,-0.466711,0.025561,1.444212,-0.104952,-0.111789
9,0.275662,0.155155,-0.248057,0.053540,0.168638,-0.495775,-0.027820,1.319162,-0.085244,-0.111390


In [23]:
# Reconstruct data from X and Y
pred = model.predict(gait)
pred.head()

H2OFrame with 151 rows and 123 columns: 


reconstr_R.ASIS.X,618.8,619.5,620.7,622.3,624.1,626.3,628.6,630.9,633.4,635.3
reconstr_R.ASIS.Y,1056.3,1054.8,1053.2,1052.4,1052.1,1052.5,1053.8,1055.5,1057.4,1059.7
reconstr_R.ASIS.Z,167.1,166.0,164.6,163.2,161.3,159.3,156.9,154.0,150.7,147.2
reconstr_L.ASIS.X,640.3,642.0,644.1,646.4,648.7,651.1,653.3,655.3,657.4,658.8
reconstr_L.ASIS.Y,1045.7,1045.3,1044.8,1044.8,1045.2,1046.1,1047.8,1049.5,1051.3,1053.3
reconstr_L.ASIS.Z,-94.9,-95.7,-96.9,-98.1,-99.6,-101.5,-103.7,-106.3,-109.4,-112.8
reconstr_V.Sacral.X,430.1,431.3,433.0,434.9,437.1,439.5,441.9,444.2,446.6,448.4
reconstr_V.Sacral.Y,1056.6,1054.8,1053.0,1051.7,1050.9,1050.8,1051.4,1052.3,1053.5,1055.0
reconstr_V.Sacral.Z,26.6,24.5,22.2,20.2,18.0,15.8,13.7,11.4,8.9,6.4
reconstr_R.Thigh.Upper.X,510.8,510.2,510.8,513.0,517.1,523.1,531.0,540.8,552.0,564.2
reconstr_R.Thigh.Upper.Y,741.4,740.0,738.8,738.0,737.6,737.6,737.9,738.7,739.4,740.3


In [24]:
# Plot original and reconstructed L.Acromium.X over time
lacro_np = np.array(h2o.as_list(gait["L.Acromium.X"]))
lacro_pred_np = np.array(h2o.as_list(pred["reconstr_L.Acromium.X"]))
line_orig = plt.plot(time_np, lacro_np, '-')
line_imp = plt.plot(time_np, lacro_pred_np, '-')

plt.title("Position of Left Acromium over Time")
plt.xlabel("Time")
plt.ylabel("X-Coordinate of Left Acromium")
blue_patch = mpatches.Patch(color = 'blue', label = 'Original')
green_patch = mpatches.Patch(color = 'green', label='Imputed')
plt.legend([blue_patch, green_patch], ["Original", "Imputed"])
plt.show()

In [25]:
# Import walking gait data with missing values
gait_miss = h2o.upload_file(_locate("smalldata/glrm_test/subject01_walk1_miss15.csv"))
gait_miss.describe()


Parse Progress: [##################################################] 100%
Uploaded pyf709d91e-fe9b-4509-a6a1-f0bd0f9d62b8 into cluster with 151 rows and 124 cols
Rows: 151 Cols: 124

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C2S,2-Byte Fractions,1,0.8,386 B,0.5
C4S,4-Byte Fractions,123,99.19355,82.6 KB,99.54594



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.38:54321,83.0 KB,151.0,1.0,124.0
mean,83.0 KB,151.0,1.0,124.0
min,83.0 KB,151.0,1.0,124.0
max,83.0 KB,151.0,1.0,124.0
stddev,0 B,0.0,0.0,0.0
total,83.0 KB,151.0,1.0,124.0



Column-by-Column Summary:



,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.6,107.79163,600.01556,1029.3,-152.5,399.40009,1034.9,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.3,550.69415,684.75494,-133.588,435.01773,627.49988,-170.9,319.34705,445.08429,157.69023,376.44724,403.25574,102.54192,295.04089,368.36661,165.6133,341.2023,438.80435,-173.0,365.72482,375.95242,-121.0,304.46347,393.67728,-173.4,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,60.01719,-74.0361,211.85968,70.39124,-69.30095,185.39088,48.22646,-140.1,254.42706,36.65646,-88.18266,627.38177,1342.9,-35.20795,511.80032,1427.4,164.48206,551.72424,1453.6,-249.9,485.94559,1227.9,216.05853,531.12134,1217.6,-266.6,424.32767,1129.5,242.4415,460.20703,1123.8,-304.8,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.6,503.10611,886.58942,-300.9,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,202.74066,36.02512,-153.1,228.04898,42.9706,-54.88903,675.97009,1562.3,44.15432,683.50897,1567.3,-94.07465,614.13971,1740.4,-19.48838
maxs,2.5,641.12048,1091.3,180.07411,661.46918,1078.0,-80.82956,451.5914,1077.8,55.29718,692.89514,788.11365,216.86717,766.44531,750.72375,136.25021,726.57483,672.42242,209.89946,674.53168,781.90839,-138.1,772.35175,780.81445,-66.30186,713.16107,681.38623,-123.1844,719.69702,531.05676,200.56787,799.47015,457.5513,154.56749,751.88885,458.64368,202.59061,764.30676,494.35605,-116.5,821.66901,425.57733,-63.825,750.77588,472.92294,-125.0268,812.50183,320.70309,108.161,935.61414,197.97754,85.62352,934.44775,209.43217,188.70589,1071.8,179.28133,153.59363,821.7641,321.80249,-34.92287,971.31763,178.00603,-6.83138,955.73096,184.36398,-1

In [26]:
# Basic GLRM using quadratic loss and no regularization

model2 = H2OGeneralizedLowRankEstimator(k=10, init="SVD", svd_method = "GramSVD", loss="Quadratic", regularization_x="None", regularization_y="None", max_iterations=2000, min_step_size=1e-6)
model2.train(x=range(1,gait_miss.ncol), training_frame=gait_miss, validation_frame=gait)
model2.show()


glrm Model Build Progress: [##################################################] 100%
Model Details
H2OGeneralizedLowRankEstimator :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1445562467796_8

Model Summary:


,number_of_iterations,final_step_size,final_objective_value
,2000.0,0.0,122299.7




ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 122299.726469
Misclassification Error (Categorical): 0.0

ModelMetricsGLRM: glrm
** Reported on validation data. **

MSE: NaN
Sum of Squared Error (Numeric): 153530.207952
Misclassification Error (Categorical): 0.0

Scoring History:


,timestamp,duration,iteration,step_size,objective
,2015-10-22 18:13:52,0.352 sec,0.0,0.7,4709528.9
,2015-10-22 18:13:52,0.360 sec,1.0,0.4,4709528.9
,2015-10-22 18:13:52,0.368 sec,2.0,0.2,4709528.9
,2015-10-22 18:13:52,0.376 sec,3.0,0.1,4709528.9
,2015-10-22 18:13:52,0.384 sec,4.0,0.0,4709528.9
---,---,---,---,---,---
,2015-10-22 18:14:19,28.280 sec,1995.0,0.0,122316.1
,2015-10-22 18:14:19,28.300 sec,1996.0,0.0,122310.5
,2015-10-22 18:14:19,28.322 sec,1997.0,0.0,122306.5
,2015-10-22 18:14:20,28.342 sec,1998.0,0.0,122303.0


In [27]:
# Plot objective function value each iteration
model2_score = model2.score_history()
plt.xlabel("Iteration")
plt.ylabel("Objective")
plt.title("Objective Function Value per Iteration")
plt.plot(model2_score["iteration"], model2_score["objective"])
plt.show()

In [28]:
# Impute missing data from X and Y
pred2 = model2.predict(gait_miss)
pred2.head()

H2OFrame with 151 rows and 123 columns: 


reconstr_R.ASIS.X,615.6,618.0,619.3,620.8,622.9,625.4,628.9,631.1,633.0,635.4
reconstr_R.ASIS.Y,1055.7,1054.9,1053.2,1052.3,1051.3,1051.5,1052.5,1054.2,1056.5,1058.8
reconstr_R.ASIS.Z,165.4,163.8,162.3,160.8,159.1,157.5,155.3,152.1,149.2,145.7
reconstr_L.ASIS.X,637.7,640.7,643.1,645.0,647.5,650.4,653.7,655.8,657.3,659.1
reconstr_L.ASIS.Y,1045.1,1045.7,1044.8,1044.7,1044.1,1045.1,1046.4,1048.0,1050.3,1052.4
reconstr_L.ASIS.Z,-96.1,-98.4,-99.3,-100.5,-101.8,-103.6,-105.6,-108.5,-111.3,-114.7
reconstr_V.Sacral.X,428.0,430.4,432.5,434.2,436.7,439.1,442.6,444.8,446.6,448.8
reconstr_V.Sacral.Y,1054.5,1054.9,1052.8,1051.6,1049.9,1049.8,1050.0,1050.8,1052.3,1053.7
reconstr_V.Sacral.Z,22.9,20.0,17.5,15.7,13.8,11.8,10.1,7.2,5.2,2.7
reconstr_R.Thigh.Upper.X,512.6,510.6,511.1,514.0,518.1,522.9,531.5,541.3,551.6,564.1
reconstr_R.Thigh.Upper.Y,742.0,741.6,740.1,739.3,738.5,737.7,738.0,738.6,739.4,740.2


In [29]:
# Plot original and imputed L.Acromium.X over time
lacro_pred_np2 = np.array(h2o.as_list(pred2["reconstr_L.Acromium.X"]))
plt.plot(time_np, lacro_np, 'b-')
plt.plot(time_np, lacro_pred_np2, 'g-')

# Mark points where training data contains missing values
idx_miss = gait_miss["L.Acromium.X"].isna().where()
plt.plot(time_np[idx_miss], lacro_np[idx_miss], "o", marker = "x", ms = 8, mew = 1.5, mec = "r")

plt.title("Position of Left Acromium over Time")
plt.xlabel("Time")
plt.ylabel("X-Coordinate of Left Acromium")
blue_patch = mpatches.Patch(color = 'blue', label = 'Original')
green_patch = mpatches.Patch(color = 'green', label = 'Imputed')
red_patch = mpatches.Patch(color = 'red', label = "Missing")
plt.legend([blue_patch, green_patch, red_patch], ["Original", "Imputed", "Missing"])
plt.show()